# Correct implementation of Homework 5 problem

Hereby I'll be presenting a possible correct interpretation of the homework 5 problem

Instructions

>Starting from the implementation contained within the notebook `05-pruning.ipynb`, extend the  `magnitude_pruning` function to allow for incremental (iterative) pruning. In the current case, if you try pruning one more time, you'll notice that it will not work as there's no way to communicate to the future calls of `magnitude_pruning` to ignore the parameters which have already been pruned. Find a way to enhance the routine s.t. it can effectively prune networks in a sequential fashion (i.e., if we passed an MLP already pruned of 20% of its parameters, we want to prune *another* 20% of parameters).
Hint: make use the mask.

In the following cells, I will make use of a very simple example which can be quickly used to (a) present the problem of sequential magnitude pruning and (b) disproof many implementation that I have seen during the corrections.

In [2]:
import torch

The two following routines create a very simple network composed of:
* a Linear layer (with weight and bias) with 10 incoming and 10 exiting neurons
* a BatchNorm layer (which is composed of two sets of parameters: the running mean and running sd used to track the stats of the incoming batch of data)

Since the net is defined as a `Sequential`, we can access its parameters by subscripting it and call the proper `.weight` or `.bias` attribute.

The net is defined by setting a manual seed to allow for reproducibility, while the weight of the Linear layer are altered by applying `relu` to set some of its parameters to **exactly** zero.
**Note that we suppose this neutralization is due to normal training and not previous pruning.**

The mask is just a bunch of ones but for the first row, in which some parameters are randomly set to 0 (i.e. they need to be pruned).

In [3]:
def get_net():
    torch.random.manual_seed(123)
    net = torch.nn.Sequential(
        torch.nn.Linear(10,10),
        torch.nn.BatchNorm1d(10)
    )
    net[0].weight.data = torch.nn.functional.relu(net[0].weight).detach()
    return net

def get_mask():
    mask = [
        torch.ones((10, 10)),
        torch.ones((10,)),
        torch.ones((10,)),
        torch.ones((10,))
    ]
    mask[0][0] = torch.Tensor([1,0,1,0,1,0,1,1,1,0])
    return mask

For the sake of visualizing the data, we print the first component of the params and the mask

In [9]:
net = get_net()
net[0].weight

Parameter containing:
tensor([[0.0000, 0.0105, 0.0000, 0.1193, 0.0000, 0.2318, 0.0000, 0.0000, 0.0000,
         0.1432],
        [0.0000, 0.1183, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2071, 0.0000,
         0.1015],
        [0.2236, 0.0589, 0.0865, 0.3052, 0.0000, 0.1002, 0.0000, 0.2260, 0.2526,
         0.0000],
        [0.2699, 0.1510, 0.1378, 0.1302, 0.2629, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0209],
        [0.0000, 0.0000, 0.0000, 0.2996, 0.0000, 0.0100, 0.0000, 0.0497, 0.2818,
         0.1934],
        [0.1122, 0.0687, 0.0746, 0.1222, 0.0000, 0.0000, 0.0000, 0.2699, 0.0189,
         0.0000],
        [0.0499, 0.2613, 0.0000, 0.0000, 0.0000, 0.0127, 0.0000, 0.0000, 0.0000,
         0.1481],
        [0.2776, 0.1933, 0.0000, 0.0000, 0.1313, 0.0071, 0.1296, 0.0000, 0.0000,
         0.2230],
        [0.1467, 0.0115, 0.0622, 0.0000, 0.0000, 0.0000, 0.0000, 0.2627, 0.1740,
         0.1106],
        [0.0000, 0.2440, 0.0992, 0.2188, 0.0000, 0.0670, 0.3088, 0.2127, 0.2536,
      

In [11]:
mask = get_mask()
mask[0]

tensor([[1., 0., 1., 0., 1., 0., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

We now define the routine for the magnitude pruning, which makes use of the mask (if not None) when deciding which parameters to choose from when creating the distribution of parameters to prune.
Note that, conversely to the in-class example, now the mask **does not operate pruning**. To do so, we define later a function **apply_mask**.

In [4]:
def magnitude_pruning(net, p, mask=None, params_to_prune=[]):
    flat = []

    for i, (name, par) in enumerate(net.named_parameters()):
        if any([l in name for l in params_to_prune]):
            if mask is None:
                flat.append(par.abs().flatten())
            else:
                flat.append(par[mask[i]!=0].abs().flatten())
    flat = torch.cat(flat, dim=0).sort()[0]

    position = int(p * flat.shape[0])
    thresh = flat[position]

    new_mask = []
    for name, par in net.named_parameters():
        if any([l in name for l in params_to_prune]):
            m = torch.where(par.abs() >= thresh, 1, 0)
            new_mask.append(m)
        else:
            new_mask.append(torch.ones_like(par))
    
    return new_mask

We recall the routine for calculating the percentage of ones within the mask; then, we define a routine for applying a mask to a given network

In [18]:
def pct_of_ones_in_mask(mask):
    return sum([m.sum().item() for m in mask]) / sum([m.numel() for m in mask])

def apply_mask(net, mask):
    for p, m in zip(net.parameters(), mask):
        p.data *= m

Let's see the % of ones in the preset mask

In [12]:
pct_of_ones_in_mask(mask)

0.9692307692307692

First of all, we build a "pristine" mask from the network setting a pruning rate of 50% without considering the preset mask:

In [13]:
net = get_net()
mask = magnitude_pruning(net, .5, params_to_prune=["0"])
print(number_of_ones_in_mask(mask))

0.5769230769230769


Since we're only pruning the Linear layer, it's expected that the result be more than 50%: we have 130 parameters in the network but we only prune 110 (100 weight + 10 bias), hence, we prune 55/130 parameters => 57.69% of the total weights will survive pruning.

Now, let us recover the previous preset mask. We first apply it to the network, pruning around 4% of its weights, then re-create the mask in sequential fashion, by adding a 50% prune on top of it.

In [19]:
mask = get_mask()
apply_mask(net, mask)
mask = magnitude_pruning(net, .5, params_to_prune=["0"], mask=mask)
print(number_of_ones_in_mask(mask))

0.5615384615384615


Now, the parameters in the network are a bit less than 57.69%, because we have to take into consideration that there was a preemptive application of pruning (albeit with a smaller pruning rate).

You can use this example to test whether your implementation is correct. Note that, if you used the same criteria for determining the pruning threshold:

```
position = int(p * flat.shape[0])
thresh = flat[position]
```

and populating the mask with `>=` instead of `>` (still correct, it's not an error!), the results should be *exactly* the same as far as % of ones.

To reuse this example, plug your version of `magnitude_pruning`, adapt this notebook considering whether or not you apply the mask within `magnitude_pruning`, adjust the order of the arguments, and see if it works correctly.